## 민규사원님 이랑 찾아볼 것.
### 1. OpenCV의 VideoCapture는 RGB or BGR 둘중에 뭔가?
bgr 가능성 농후

### 2. BGR input으로 fear가 많았을 테니, RGB input으로 바꿔서 진행하면 ouput이 다를까?
하는 중


## 흥용 책임님 요청사항
에러난 파일 3~6번 사이를 앞쪽이든 뒷쪽이든 빼놔라

In [1]:
import cv2
import numpy as np
from PIL import Image

In [8]:

img = cv2.imread('/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png')
bgr = cv2.imread('/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png', cv2.IMREAD_COLOR)
bgr = cv2.imread('/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png', cv2.COLOR_RGB2BGR)
pil = Image.open('/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png')


cv2.imshow('img', img)
cv2.imshow('bgr', bgr)
pil.show()


cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
img = '/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png'

cap = cv2.VideoCapture(img)
ret, frame = cap.read()

# cv2.imshow('frame', frame)
bgrframe = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


cv2.imshow('frame', frame)
cv2.imshow('bgrframe', bgrframe)
cv2.imshow('rgbframe', rgbframe)


cv2.waitKey(0)
cv2.destroyAllWindows()


In [25]:
img = '/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png'

cap = cv2.VideoCapture(img)
ret, frame = cap.read()

# cv2.imshow('frame', frame)
bgrframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# rgbframe = cv2.cvtColor(bgrframe, cv2.COLOR_BGR2RGB)


cv2.imshow('frame', frame)
cv2.imshow('bgrframe', bgrframe)
# cv2.imshow('rgbframe', rgbframe)


cv2.waitKey(0)
cv2.destroyAllWindows()


# zorang2 Code


In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from emotion import Emotion
import mediapipe as mp
import csv
import glob
import re
from tqdm import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-5wxooxps because the default path (/home/sldev1/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2023-01-20 15:00:39.153489: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sldev1/anaconda3/envs/dms2/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.0/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-01-20 15:00:39.153514: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dirpath = "/media/sldev1/T7/DMS2/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.mp4')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))
filepath = filepath[7:9]
print(filepath)
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

['/media/sldev1/T7/DMS2/08. 정승아_1214.mp4', '/media/sldev1/T7/DMS2/09. 류채원_1214.mp4']


2023-01-20 15:00:39.898100: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-01-20 15:00:39.921105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-20 15:00:39.921360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.835GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2023-01-20 15:00:39.921401: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-20 15:00:39.921627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 00

## 기존 code

In [36]:
from PIL import Image

for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
                rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # bgr로 들어감. 확인완료
                results = face_detection.process(rgbframe)
                rgbframe.flags.writeable = True
                rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)
                cv2.imshow('rgbframe', rgbframe)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = rgbframe.shape[0]*bbox.ymin
                    bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
                    left = rgbframe.shape[1]*bbox.xmin
                    right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

#                     pre_emo, one, two = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
                    pre_emo, one, two = emo.process(rgbframe[int(top)-10:int(bottom)+10, int(left)-10:int(right)+10])
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
                    cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)


                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()
                
                if cap.get(1) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
            cap.release()
            cv2.destroyAllWindows()


# 1258 frame에서 멈추는 중

  0%|                                                     | 0/2 [00:00<?, ?it/s]

/media/sldev1/T7/DMS2/08. 정승아_1214.mp4


 50%|██████████████████████▌                      | 1/2 [00:10<00:10, 10.72s/it]

/media/sldev1/T7/DMS2/09. 류채원_1214.mp4


100%|█████████████████████████████████████████████| 2/2 [00:11<00:00,  5.83s/it]


## RGB로 input 넣기(내가수정)


In [4]:
from PIL import Image

for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_rgb/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
#                 cv2.imshow('frame',frame)
#                 rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(frame)
                frame.flags.writeable = False
#                 rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = frame.shape[0]*bbox.ymin
                    bottom = frame.shape[0]*(bbox.xmin+bbox.height)
                    left = frame.shape[1]*bbox.xmin
                    right = frame.shape[1]*(bbox.xmin+bbox.width)

                    pre_emo, one, two = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
#                     pre_emo, one, two = emo.process(rgbframe[int(top)-100:int(bottom)+50, int(left)-50:int(right)+50])
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
                    cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)

#                     cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)
#                     cv2.rectangle(frame, (int(left)-50, int(top)-100), (int(right)+50, int(bottom)+50), (255, 255, 0), 3)
                    cv2.rectangle(frame, (int(left)-10, int(top)-10), (int(right)+10, int(bottom)+10), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_rgb/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()
                
                if cap.get(1) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
            cap.release()
            cv2.destroyAllWindows()



            # 1258 frame에서 멈추는 중

  0%|                                                     | 0/2 [00:00<?, ?it/s]

/media/sldev1/T7/DMS2/08. 정승아_1214.mp4


 50%|█████████████████████▌                     | 1/2 [32:38<32:38, 1958.08s/it]

/media/sldev1/T7/DMS2/09. 류채원_1214.mp4


100%|█████████████████████████████████████████| 2/2 [1:18:53<00:00, 2366.94s/it]


In [13]:
for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:
        while cap.isOpened():
            success, image = cap.read()
            
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            cv2.imshow('image', image)
            results = face_detection.process(image)

            # Draw the face detection annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.detections:
                for detection in results.detections:
                    mp_drawing.draw_detection(image, detection)


            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Face Detection', image)


            if cv2.waitKey(1) == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

  0%|                                                     | 0/2 [00:00<?, ?it/s]

/media/sldev1/T7/DMS2/08. 정승아_1214.mp4


 50%|██████████████████████▌                      | 1/2 [00:09<00:09,  9.85s/it]

/media/sldev1/T7/DMS2/09. 류채원_1214.mp4


100%|█████████████████████████████████████████████| 2/2 [00:17<00:00,  8.78s/it]


## 01/13(금) 오류

In [ ]:
  6%|████▌                                                                      | 3/50 [2:52:17<45:43:23, 3502.20s/it]
/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4
[h264 @ 0x559e98b11500] Invalid NAL unit size (7082 > 1541).
[h264 @ 0x559e98b11500] Error splitting the input into NAL units.
  6%|████▌                                                                      | 3/50 [2:54:15<45:30:00, 3485.13s/it]
---------------------------------------------------------------------------
error                                     Traceback (most recent call last)
/tmp/ipykernel_17891/1244803589.py in <module>
     12             while(cap.isOpened()):
     13                 ret, frame = cap.read()
---> 14                 rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
     15                 results = face_detection.process(rgbframe)
     16                 rgbframe.flags.writeable = True

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
                
# Frame 2566에서 에러뜸.
# 다시 하니까 또 됨. 머노?

## 01/18 (수) csv file editing

In [1]:
pwd

'/home/sldev1/github/TIL/Internship/DMS2_SMU'

In [12]:
import glob
import re
import csv

In [13]:
dirpath = "./FER_Result_SL/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.csv')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[0]))
filepath = filepath[0:2]
print(filepath)


['./FER_Result_SL/01. 김형민_1201.csv', './FER_Result_SL/02. 김두용_1205.csv']


In [15]:
v = open(filepath[0])
r = csv.reader(v)
row0 = r.next()
row0.append('Valence')
print(row0)

AttributeError: '_csv.reader' object has no attribute 'next'